In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.datasets import make_classification
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import shap

#### Load data

In [3]:
df = pd.read_csv('/Users/lindan/Dropbox/PhD/Projects/PLF/FTIR/Data/JM006_0901_whole.csv')
df = df.dropna(subset=['milkweightlbs_sca'])
df = df.dropna(subset=['cells'])
df_f = df[df['disease'] == 0]
# df_d = df[df['disease'] == 1][df['disease_in'] > 14] # 210 
# df_d = df[df['disease'] == 1][df['disease_in'] <= 14][df['disease_in'] >= 11] # 94
# df_d = df[df['disease'] == 1][df['disease_in'] <= 10][df['disease_in'] >= 8] # 136
# df_d = df[df['disease'] == 1][df['disease_in'] <= 7][df['disease_in'] >= 6] # 125
# df_d = df[df['disease'] == 1][df['disease_in'] <= 5][df['disease_in'] >= 4] # 199
# df_d = df[df['disease'] == 1][df['disease_in'] == 3] # 114
# df_d = df[df['disease'] == 1][df['disease_in'] == 2] # 117
df_d = df[df['disease'] == 1][df['disease_in'] == 1] # 147
df = pd.concat([df_f, df_d])
df = df.reset_index(drop=True)
X = np.hstack((df.iloc[:,1:488].values, df['milkweightlbs_sca'].values.reshape(-1, 1), df['cells_sca'].values.reshape(-1, 1), df['parity_sca'].values.reshape(-1, 1)))
y = df['disease']

Boolean Series key will be reindexed to match DataFrame index.


#### PLS-DA

In [5]:
# Calculate Variable Importance in Projection (VIP) in PLS-DA
def calculate_vip(model):
    t = model.x_scores_
    w = model.x_weights_
    q = model.y_loadings_
    p, h = w.shape
    vips = np.zeros((p,))
    s = np.diag(np.matmul(np.matmul(np.matmul(t.T,t),q.T), q)).reshape(h, -1)
    total_s = np.sum(s)
    for i in range(p):
        weight = np.array([ (w[i,j] / np.linalg.norm(w[:,j]))**2 for j in range(h) ])
        vips[i] = np.sqrt(p*(np.matmul(s.T, weight))/total_s)
    return vips

# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit PLS-DA
    model = PLSRegression(n_components=2)
    model.fit(X_sampled, y_sampled)

    # Calculate VIP
    VIP = calculate_vip(model)
    i = i + 1
    imp[f'Fea_{i}'] = VIP

imp_pls_da = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'PLS-DA'
})
imp_pls_da

,variable,imp,model
0,1002.91,1.402973,PLS-DA
1,1006.77,1.356892,PLS-DA
2,1010.63,1.451879,PLS-DA
3,1014.48,1.448652,PLS-DA
4,1018.34,1.492027,PLS-DA
...,...,...,...
485,2993.3,0.859332,PLS-DA
486,2997.16,0.941502,PLS-DA
487,milkweightlbs,2.241255,PLS-DA
488,cells,1.307038,PLS-DA


#### Lasso

In [26]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit Lasso
    model = LogisticRegression(penalty='l1', solver='saga', C=1.0)
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    coefficients = model.coef_[0]
    coefficients = coefficients.flatten().tolist()
    i = i + 1
    imp[f'Fea_{i}'] = coefficients

imp_lasso = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1), 
    'model': 'Lasso'
})
imp_lasso

The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the

,variable,imp,model
0,1002.91,0.000000,Lasso
1,1006.77,0.003193,Lasso
2,1010.63,0.004821,Lasso
3,1014.48,0.000000,Lasso
4,1018.34,0.000000,Lasso
...,...,...,...
485,2993.3,0.003509,Lasso
486,2997.16,0.001532,Lasso
487,milkweightlbs,1.032065,Lasso
488,cells,0.429457,Lasso


#### Ridge

In [27]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit Ridge
    model = RidgeClassifier()
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    coefficients = model.coef_[0]
    coefficients = coefficients.flatten().tolist()
    i = i + 1
    imp[f'Fea_{i}'] = coefficients

imp_Ridge = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'Ridge'
})
imp_Ridge

,variable,imp,model
0,1002.91,0.052654,Ridge
1,1006.77,0.002649,Ridge
2,1010.63,0.036501,Ridge
3,1014.48,0.017925,Ridge
4,1018.34,0.011952,Ridge
...,...,...,...
485,2993.3,0.008757,Ridge
486,2997.16,0.090833,Ridge
487,milkweightlbs,0.355259,Ridge
488,cells,0.063060,Ridge


#### Elastic Net

In [28]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit EN
    model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    coefficients = model.coef_[0]
    coefficients = coefficients.flatten().tolist()
    i = i + 1
    imp[f'Fea_{i}'] = coefficients

imp_Elastic_Net = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'EN'
})
imp_Elastic_Net

The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the

,variable,imp,model
0,1002.91,6.877237e-04,EN
1,1006.77,6.846354e-03,EN
2,1010.63,9.510051e-03,EN
3,1014.48,8.185471e-04,EN
4,1018.34,1.690693e-09,EN
...,...,...,...
485,2993.3,8.052354e-03,EN
486,2997.16,1.225271e-02,EN
487,milkweightlbs,1.014428e+00,EN
488,cells,3.961107e-01,EN


#### Random forest

In [10]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit RF
    model = RandomForestClassifier(max_depth=2, n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    feature_importances = model.feature_importances_
    coefficients = feature_importances.flatten().tolist()
    i = i + 1
    imp[f'Fea_{i}'] = coefficients

imp_RF = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'RF'
})
imp_RF

,variable,imp,model
0,1002.91,0.003100,RF
1,1006.77,0.002860,RF
2,1010.63,0.001704,RF
3,1014.48,0.001903,RF
4,1018.34,0.003670,RF
...,...,...,...
485,2993.3,0.001007,RF
486,2997.16,0.000402,RF
487,milkweightlbs,0.011055,RF
488,cells,0.006357,RF


#### XGBoost

In [11]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit XGBoost
    model = XGBClassifier(max_depth=2, n_estimators=100, use_label_encoder=False, eval_metric='logloss')
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    feature_importances = model.feature_importances_
    coefficients = feature_importances.flatten().tolist()
    i = i + 1
    imp[f'Fea_{i}'] = coefficients

imp_XGBoost = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'XGBoost'
})
imp_XGBoost

,variable,imp,model
0,1002.91,0.001850,XGBoost
1,1006.77,0.001615,XGBoost
2,1010.63,0.004022,XGBoost
3,1014.48,0.001579,XGBoost
4,1018.34,0.003065,XGBoost
...,...,...,...
485,2993.3,0.001602,XGBoost
486,2997.16,0.003005,XGBoost
487,milkweightlbs,0.012999,XGBoost
488,cells,0.013095,XGBoost


#### MLP

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam

class MLPClassifierCustom(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, hidden_layer_sizes=(100,), activation='relu', epochs=200, batch_size=32):
        self.input_shape = input_shape  
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(self.hidden_layer_sizes[0], activation=self.activation, input_shape=(self.input_shape,)))
        
        for layer_size in self.hidden_layer_sizes[1:]:
            model.add(Dense(layer_size, activation=self.activation))
            
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return (y_pred > 0.5).astype(int).flatten()

In [16]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_0 = X[y == 0]
X_1 = X[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))

    # Fit ANN
    model = MLPClassifierCustom(input_shape=X.shape[1], hidden_layer_sizes=(32, 128, 32)) 
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    explainer = shap.GradientExplainer(model.model, [X_sampled, y_sampled])
    shap_values = explainer.shap_values(X_sampled)  
    shap_values = np.array(shap_values)
    shap_values = shap_values.reshape(X_sampled.shape[0], X_sampled.shape[1])
    shap_values = [abs(x) for x in np.mean(abs(shap_values), axis=0)]
    i = i + 1
    imp[f'Fea_{i}'] = shap_values

imp_ANN = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'MLP'
})
imp_ANN

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` arg

,variable,imp,model
0,1002.91,0.002659,MLP
1,1006.77,0.002047,MLP
2,1010.63,0.002325,MLP
3,1014.48,0.001539,MLP
4,1018.34,0.002655,MLP
...,...,...,...
485,2993.3,0.000877,MLP
486,2997.16,0.001136,MLP
487,milkweightlbs,0.093873,MLP
488,cells,0.044710,MLP


#### CNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam

class CNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, epochs=200, batch_size=32):
        self.input_shape = input_shape  
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Conv1D(8, kernel_size=3, activation='relu', input_shape=self.input_shape)) # 16
        model.add(Conv1D(12, kernel_size=3, activation='relu')) 
        model.add(Flatten())
        model.add(Dense(16, activation='relu')) # 16
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return (y_pred > 0.5).astype(int).flatten()

In [19]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_cnn = X.reshape(X.shape[0], X.shape[1], 1)
X_0 = X_cnn[y == 0]
X_1 = X_cnn[y == 1]
n_samples = 50 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit CNN
    model = CNNClassifier(input_shape=(X_cnn.shape[1], 1))
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    explainer = shap.GradientExplainer(model.model, [X_sampled, y_sampled])
    shap_values = explainer.shap_values(X_sampled)  
    shap_values = np.array(shap_values)
    shap_values = shap_values.reshape(X_sampled.shape[0], X_sampled.shape[1])
    shap_values = [abs(x) for x in np.mean(abs(shap_values), axis=0)]
    i = i + 1
    imp[f'Fea_{i}'] = shap_values

imp_CNN = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'CNN'
})
imp_CNN

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` arg

,variable,imp,model
0,1002.91,0.000886,CNN
1,1006.77,0.001341,CNN
2,1010.63,0.001762,CNN
3,1014.48,0.001902,CNN
4,1018.34,0.001783,CNN
...,...,...,...
485,2993.3,0.001529,CNN
486,2997.16,0.001480,CNN
487,milkweightlbs,0.100885,CNN
488,cells,0.035565,CNN


#### LSTM

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, MaxPooling1D
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam

class LSTMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, units=20, epochs=200, batch_size=32):
        self.input_shape = input_shape  
        self.units = units 
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(LSTM(self.units, activation='relu', input_shape=self.input_shape))
        model.add(Dense(64, activation='relu')) # 64
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    
    def predict_proba(self, X):
        return self.model.predict(X)
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return (y_pred > 0.5).astype(int).flatten()

In [23]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_lstm = X.reshape(X.shape[0], 1, X.shape[1])
X_0 = X_lstm[y == 0]
X_1 = X_lstm[y == 1]
n_samples = 5 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit LSTM
    model = LSTMClassifier(input_shape=(1, X_lstm.shape[2]), units=20, epochs=200, batch_size=32)
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    explainer = shap.GradientExplainer(model.model, [X_sampled, y_sampled])
    shap_values = explainer.shap_values(X_sampled)  
    shap_values = np.array(shap_values)
    shap_values = shap_values.reshape(X_sampled.shape[0], X_sampled.shape[2])
    shap_values = [abs(x) for x in np.mean(abs(shap_values), axis=0)]
    i = i + 1
    imp[f'Fea_{i}'] = shap_values

imp_LSTM = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] + ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'LSTM'
})
imp_LSTM

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` arg

,variable,imp,model
0,1002.91,0.001651,LSTM
1,1006.77,0.000897,LSTM
2,1010.63,0.001881,LSTM
3,1014.48,0.000945,LSTM
4,1018.34,0.001571,LSTM
...,...,...,...
485,2993.3,0.000607,LSTM
486,2997.16,0.001872,LSTM
487,milkweightlbs,0.127211,LSTM
488,cells,0.059509,LSTM


#### GRU

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam

class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, units=20, epochs=200, batch_size=32):
        self.input_shape = input_shape  
        self.units = units  
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(GRU(self.units, activation='relu', input_shape=self.input_shape))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    
    def predict_proba(self, X):
        return self.model.predict(X)
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return (y_pred > 0.5).astype(int).flatten()

In [25]:
# Construct feature importance dataframe
imp = pd.DataFrame()
i = 0

# Down samping
X_gru = X.reshape(X.shape[0], 1, X.shape[1])
X_0 = X_gru[y == 0]
X_1 = X_gru[y == 1]
n_samples = 5 

for i in range(n_samples):
    sampled_indices = np.random.choice(len(X_0), size=len(X_1), replace=True)
    X_0_sampled = X_0[sampled_indices]
    X_sampled = np.vstack((X_0_sampled, X_1))
    y_sampled = np.array([0] * len(X_0_sampled) + [1] * len(X_1))
    
    # Fit GRU
    model = GRUClassifier(input_shape=(1, X_gru.shape[2]), units=20, epochs=200, batch_size=32)
    model.fit(X_sampled, y_sampled)

    # Calculate feature importance
    explainer = shap.GradientExplainer(model.model, [X_sampled, y_sampled])
    shap_values = explainer.shap_values(X_sampled)  
    shap_values = np.array(shap_values)
    shap_values = shap_values.reshape(X_sampled.shape[0], X_sampled.shape[2])
    shap_values = [abs(x) for x in np.mean(abs(shap_values), axis=0)]
    i = i + 1
    imp[f'Fea_{i}'] = shap_values

imp_GRU = pd.DataFrame({
    'variable': df.iloc[:,1:488].columns.tolist() + ['milkweightlbs'] + ['cells'] +  ['parity'],
    'imp': imp.mean(axis=1),
    'model': 'GRU'
})
imp_GRU

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` arg

,variable,imp,model
0,1002.91,0.003386,GRU
1,1006.77,0.002044,GRU
2,1010.63,0.002614,GRU
3,1014.48,0.001548,GRU
4,1018.34,0.001523,GRU
...,...,...,...
485,2993.3,0.001312,GRU
486,2997.16,0.001361,GRU
487,milkweightlbs,0.117175,GRU
488,cells,0.051612,GRU
